In [1]:
#@markdown Importowanie bibliotek
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import scipy.optimize as opt
import multiprocessing
import scipy.integrate as integrate


In [2]:
np.random.seed(1)
#ustawianie parametrów
N = 10**6
T = 1
mu = 0.06
r = 0.05
sigma = 0.25
s0 = 100
K = 110
disc = np.exp(-r *T)
X0 = 100
muX = 0.06
sigmaX = 0.25
rho = 0.9
m = 0.001

In [3]:
#@title Generowanie trajektorii aktywa

nsteps = 250
def gen_traj(n=nsteps, T= T, s0 = s0, mu = mu, sigma = sigma):
  w = np.random.normal(loc=0,scale=1,size=n)*T**0.5/n**0.5
  w = np.cumsum(w)
  ts = np.arange(n)/n * T
  doexpa = (mu- sigma**2/2)*ts + sigma*w 
  trajektoria = np.hstack([s0, s0 * np.exp(doexpa)])
  return trajektoria


n = 500000
trajs = []
for i in range(n):
  trajs.append(gen_traj())
trajs=np.array(trajs)
  

In [4]:
#@title Robienie kraty, na poczatek co 1 na kazdym do 300
A=np.arange(0,300)
B=np.arange(0,300)
for i in range(249):
  A=np.vstack((A,B))
A=np.transpose(A)
A[0,1]


0

In [5]:
#@title Wyznaczanie prawdopodobieństw
probs = np.zeros((300,300))
visits = np.zeros((300))
trajs[trajs>299]=299
int_trajs = np.rint(trajs)
for i in range(249):
  Ai = int_trajs[:,i]
  Aii = int_trajs[:,i+1]
  #print(i)
  for j in range(len(Ai)):
    visits[int(Ai[j])]+=1
    probs[int(Ai[j]),int(Aii[j])]+=1
#probs = np.divide(probs, visits, out=np.zeros_like(probs), where=visits!=0)
  



In [26]:
visits[100]

4279542.0

In [42]:
#@title upewnianko ze dziala
suma = []

for i in range(300): 
  sw = 0
  for j in range(300):
    sw+=  probs[i,j]#/max(1,visits[i])
  suma.append(sw)
#suma
#okej, chodzi o to ze niektorych malych wartosci nawet nie osiagamy wiec to sie dodaje do pelnej liczby wszedzie, poza pierwszymi ilomas tam rzeczami


In [7]:
#@title unormowanie pstwa
for i in range(300): 
  for j in range(300):
    probs[i,j]=  probs[i,j]/max(1,visits[i])
  #suma.append(sw)

In [32]:
#@title Funkcja payoffu
#tutaj later zrob jakies inne opcje


In [8]:
#@title Funkcje na ceny klasycznych opcji  
def d1(S, sigma, K, t, T, r):
  return (np.log(S/K) + (r + sigma**2/2)*(T-t))/(sigma * np.sqrt(T-t))

def d2(S, sigma, K, t, T, r):
  return d1(S, sigma, K, t, T, r) - sigma * np.sqrt(T-t)

def BS(s0, K, T, r, sigma, typ = 'call', t = 0):
  call = s0 * norm.cdf(d1(s0, sigma, K, t, T, r)) - np.exp(-r*(T - t)) * K * norm.cdf(d2(s0, sigma, K, t, T, r))
  if typ == 'call':
    return call
  elif typ == 'put':
    return call - s0 + K * np.exp(-r*(T-t))

def binary_price(St, K, T, r, sigma, typ = 'call', t = 0):
  if typ == 'call':
    return np.exp(-r*(T-t)) * norm.cdf(d2(St, sigma, K, t, T, r))
  else: 
    return np.exp(-r*(T-t)) * norm.cdf(- d2(St, sigma, K, t, T, r))

#funcje na delty klasycznych opcji 

def delta(St, K, T, r, sigma, typ = 'call', t = 0): 
  if typ == 'call':
    return norm.cdf(d1(St, sigma, K, t, T, r))
  else:
    return (norm.cdf(d1(St, sigma, K, t, T, r)) -1)

def delta_binary(St, K, T, r, sigma, typ = 'call', t = 0):
  db = np.exp(-r*(T-t) - d2(St, sigma, K, t, T, r)**2/2)/(St*sigma * np.sqrt((T-t) * 2 * np.pi))
  if typ == 'call':
    return db
  else:
    return -db


def geom_brown(t, mu, sigma, w): # wzór na geometryczny ruch browna 
  return np.exp((mu - (sigma**2)/2)*t + sigma*w)
print(BS(s0=100,K=110,T=1,r=r,sigma=sigma,typ="call",t=0))
print(BS(s0=100,K=110,T=1,r=r,sigma=sigma,typ="put",t=0))

8.026384693853352
12.661621388931891


In [15]:
#@title Wycena
def SSP_call( K=110):
  Grid = np.zeros((300,250))
  values = np.arange(0,300)
  Grid[:,-1] = (values-K)*(values-K>0)*np.exp(-r*T)
  length = 250 #biore sobie jakas przykladowa zeby na niej moze pozniej pracowac
  for t in np.arange(length-2,-1,-1):
    for j in range(300):
      payoff = (j-K)*(j-K>0)*np.exp(-r*t/length)
      prob = probs[j,:]
      V = prob*Grid[:,t+1]*np.exp(-r/length)
      EV = sum(V)
      Grid[j,t] = max(EV, payoff) 
  return Grid[100,0]
SSP_call()


8.01723366273143

In [14]:
def SSP_put( K=110):
  Grid = np.zeros((300,250))
  values = np.arange(0,300)
  Grid[:,-1] = (K-values)*(K-values>0)*np.exp(-r*T)
  length = 250 #biore sobie jakas przykladowa zeby na niej moze pozniej pracowac
  for t in np.arange(length-2,-1,-1):
    for j in range(300):
      payoff = (K-j)*(K-j>0)*np.exp(-r*t/length)
      prob = probs[j,:]
      V = prob*Grid[:,t+1]*np.exp(-r/length)
      EV = sum(V)
      Grid[j,t] = max(EV, payoff) 
  return Grid[100,0]
SSP_put()

13.48244246421544

In [18]:
def SSP_put_barierowa( K=110,r=0.05,T=1):
  Grid = np.zeros((300,250))
  values = np.arange(0,300)
  Grid[:,-1] = (K-values>0)*np.exp(-r*T)
  length = 250 #biore sobie jakas przykladowa zeby na niej moze pozniej pracowac
  for t in np.arange(length-2,-1,-1):
    for j in range(300):
      payoff = (K-j>0)*np.exp(-r*t/length)
      prob = probs[j,:]
      V = prob*Grid[:,t+1]*np.exp(-r/length)
      EV = sum(V)
      Grid[j,t] = max(EV, payoff) 
  return Grid[120,0]
SSP_put_barierowa()

0.6547283937210666

In [16]:
def SSP_call_barierowa( K=110):
  Grid = np.zeros((300,250))
  values = np.arange(0,300)
  Grid[:,-1] = (values-K>0)*np.exp(-r*T)
  length = 250 #biore sobie jakas przykladowa zeby na niej moze pozniej pracowac
  for t in np.arange(length-2,-1,-1):
    for j in range(300):
      payoff = (j-K>0)*np.exp(-r*t/length)
      prob = probs[j,:]
      V = prob*Grid[:,t+1]*np.exp(-r/length)
      EV = sum(V)
      Grid[j,t] = max(EV, payoff) 
  return Grid[100,0]
SSP_call_barierowa()

0.6840295703327316

In [46]:
length = 250
np.arange(length-1,0,-1)

array([249, 248, 247, 246, 245, 244, 243, 242, 241, 240, 239, 238, 237,
       236, 235, 234, 233, 232, 231, 230, 229, 228, 227, 226, 225, 224,
       223, 222, 221, 220, 219, 218, 217, 216, 215, 214, 213, 212, 211,
       210, 209, 208, 207, 206, 205, 204, 203, 202, 201, 200, 199, 198,
       197, 196, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185,
       184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 173, 172,
       171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159,
       158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146,
       145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133,
       132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120,
       119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107,
       106, 105, 104, 103, 102, 101, 100,  99,  98,  97,  96,  95,  94,
        93,  92,  91,  90,  89,  88,  87,  86,  85,  84,  83,  82,  81,
        80,  79,  78,  77,  76,  75,  74,  73,  72,  71,  70,  6

In [33]:
A=np.arange(1,5)
B=np.arange(3,7)
A*B

array([ 3,  8, 15, 24])